In [1]:
import numpy as np
import torch

In [2]:
num_slice = 64
D, H, W = 184, 630, 630

volume = np.random.rand(D, H, W)
probability = np.zeros((7, D, H, W), dtype=np.float32)
count = np.zeros((7, D, H, W), dtype=np.float32)
pad_volume = np.pad(volume, [[0, 0], [0, 640 - H], [0, 640 - W]], mode='constant', constant_values=0)
zz = list(range(0, D - num_slice, num_slice//2)) + [D - num_slice]

In [3]:
class dotdict(dict):
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

In [4]:
cfg = dotdict(
    arch ='resnet34d',
    checkpoint=\
    '/kaggle/input/hengck-czii-cryo-et-weights-01/resnet34d-aug-noise-00003956.pth',
       #  '/kaggle/input/hengck-czii-cryo-et-weights-01/resnet34d-00002300.pth',
       # '/kaggle/input/hengck-czii-cryo-et-weights-01/00003531.pth',
    threshold={ 
        'apo-ferritin': 0.05,
        'beta-amylase': 0.05,
        'beta-galactosidase': 0.05,
        'ribosome': 0.05,
        'thyroglobulin': 0.05,
        'virus-like-particle': 0.05,
    },
)


In [ ]:
for z in zz:
    print('\r',f'z:{z}', end='',flush=True)
    image = pad_volume[z:z + num_slice]
    print(image.shape)
    batch = dotdict(
        image=torch.from_numpy(image).unsqueeze(0),
    )
    with torch.amp.autocast('cuda', enabled=True):
        with torch.no_grad():
            output = net(batch)
    prob = output['particle'][0].cpu().numpy()
    probability[:, z:z + num_slice] += prob[:, :, :H, :W]
    count[:, z:z + num_slice] += 1

In [1]:
import pandas as pd
import json
import os
import numpy as np

PARTICLE= [
    {
        "name": "apo-ferritin",
        "difficulty": 'easy',
        "pdb_id": "4V1W",
        "label": 1,
        "color": [0, 255, 0, 0],
        "radius": 60,
        "map_threshold": 0.0418
    },
    {
        "name": "beta-amylase",
        "difficulty": 'ignore',
        "pdb_id": "1FA2",
        "label": 2,
        "color": [0, 0, 255, 255],
        "radius": 65,
        "map_threshold": 0.035
    },
    {
        "name": "beta-galactosidase",
        "difficulty": 'hard',
        "pdb_id": "6X1Q",
        "label": 3,
        "color": [0, 255, 0, 255],
        "radius": 90,
        "map_threshold": 0.0578
    },
    {
        "name": "ribosome",
        "difficulty": 'easy',
        "pdb_id": "6EK0",
        "label": 4,
        "color": [0, 0, 255, 0],
        "radius": 150,
        "map_threshold": 0.0374
    },
    {
        "name": "thyroglobulin",
        "difficulty": 'hard',
        "pdb_id": "6SCJ",
        "label": 5,
        "color": [0, 255, 255, 0],
        "radius": 130,
        "map_threshold": 0.0278
    },
    {
        "name": "virus-like-particle",
        "difficulty": 'easy',
        "pdb_id": "6N4V",
        "label": 6,
        "color": [0, 0, 0, 255],
        "radius": 135,
        "map_threshold": 0.201
    }
]

PARTICLE_COLOR=[[0,0,0]]+[
    PARTICLE[i]['color'][1:] for i in range(6)
]
PARTICLE_NAME=['none']+[
    PARTICLE[i]['name'] for i in range(6)
]

def read_one_truth(id, overlay_dir):
    data_list = []

    json_dir = f'{overlay_dir}/{id}/Picks'
    for p in PARTICLE_NAME[1:]:
        json_file = f'{json_dir}/{p}.json'

        with open(json_file, 'r') as f:
            json_data = json.load(f)

        num_point = len(json_data['points'])
        loc = np.array([list(json_data['points'][i]['location'].values()) for i in range(num_point)])

        for point in loc:
            data_list.append({"particle_type": p, "x": point[0], "y": point[1], "z": point[2]})

    # データフレームを作成
    df = pd.DataFrame(data_list)
    return df

valid_dir = "/home/naoya/kaggle/czii/input/czii-cryo-et-object-identification/train/overlay/ExperimentRuns"
# ディレクトリ名のみを取得
valid_ids = [
    name for name in os.listdir(valid_dir)
    if os.path.isdir(os.path.join(valid_dir, name))
]
for id in valid_ids:
    truth = read_one_truth(id, valid_dir) #=f'{valid_dir}/overlay/ExperimentRuns')
    solution = pd.DataFrame(truth)
    solution.insert(loc=0, column='id', value=np.arange(len(solution)))
    solution.insert(loc=1, column="experiment",  value=id)
    solution.to_csv(f"/home/naoya/kaggle/czii/input/solution/solution_{id}.csv", index=False)

In [ ]:
import json
copick_config_path = TRAIN_DATA_DIR + "/copick.config"

with open(copick_config_path) as f:
    copick_config = json.load(f)

copick_config['static_root'] = '/kaggle/input/czii-cryo-et-object-identification/test/static'

copick_test_config_path = 'copick_test.config'

with open(copick_test_config_path, 'w') as outfile:
    json.dump(copick_config, outfile)

In [1]:
import os
import pandas as pd

def concat_csv_files(input_dir, output_file):
    # Get list of all CSV files in the directory
    csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]
    
    # Read and concatenate all CSV files
    df_list = [pd.read_csv(os.path.join(input_dir, file)) for file in csv_files]
    concatenated_df = pd.concat(df_list, axis=0)
    
    # Save the concatenated dataframe to a new CSV file
    concatenated_df.to_csv(output_file, index=False)

# Example usage
input_directory = '/home/naoya/kaggle/czii/input/solution'
output_csv = '/home/naoya/kaggle/czii/input/solution/solution.csv'


In [1]:
import pandas as pd

df_dir = '/home/naoya/kaggle/czii/input/solution'
valid_ids = ['TS_6_4', 'TS_69_2', 'TS_6_6', 'TS_73_6']

df_list = []
for id in valid_ids:
    df = pd.read_csv(f'{df_dir}/solution_{id}.csv')
    df_list.append(df)

concatenate_df = pd.concat(df_list, axis=0)
concatenate_df.to_csv(f'{df_dir}/solution_1-4.csv', index=False)

In [6]:
import numpy as np
import os

label_dir = '/home/naoya/kaggle/czii/input/extra_data/numpy/denoised'

print(os.listdir(label_dir))

for file in os.listdir(label_dir):
    if 'label' in file:
        label = np.load(f'{label_dir}/{file}')
        unique_labels = np.unique(label)
        print(f'Unique labels in {file}: {unique_labels}')

['train_label_TS_23.npy', 'train_image_TS_0.npy', 'train_image_TS_14.npy', 'train_label_TS_12.npy', 'train_label_TS_3.npy', 'train_label_TS_21.npy', 'train_image_TS_20.npy', 'train_label_TS_14.npy', 'train_image_TS_23.npy', 'train_image_TS_2.npy', 'train_label_TS_11.npy', 'train_image_TS_6.npy', 'train_image_TS_3.npy', 'train_image_TS_24.npy', 'train_label_TS_9.npy', 'train_image_TS_25.npy', 'train_image_TS_17.npy', 'train_image_TS_8.npy', 'train_label_TS_13.npy', 'train_label_TS_5.npy', 'train_label_TS_8.npy', 'train_image_TS_21.npy', 'train_label_TS_19.npy', 'train_image_TS_15.npy', 'train_image_TS_5.npy', 'train_image_TS_18.npy', 'train_label_TS_26.npy', 'train_label_TS_7.npy', 'train_image_TS_22.npy', 'train_label_TS_2.npy', 'train_image_TS_4.npy', 'train_image_TS_7.npy', 'train_image_TS_19.npy', 'train_image_TS_16.npy', 'train_image_TS_12.npy', 'train_label_TS_17.npy', 'train_label_TS_18.npy', 'train_image_TS_10.npy', 'train_label_TS_0.npy', 'train_label_TS_22.npy', 'train_image_T